In [ ]:
# 由于我的方法运行程序的效率过低，故采推荐使用它的方案
# 此部分海带的模拟的核心代码是由chen_yu_xuan编写的，以下是他的个人主页链接
# https://space.bilibili.com/67131398
# 古镇天写了多线程优化，虽然效果不明显

from concurrent.futures import ThreadPoolExecutor, wait, ALL_COMPLETED
import numpy
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from tqdm import tqdm



# 对象参数
收割时间 = 5
公摊高度 = 2
p = 3 / 4096 * 0.14  # 每tick海带生长的概率
最大生长高度 = 25

# 时间参数
起始时间 = 1
终止时间 = 72000
时间步长 = 1

# 定义状态转移矩阵，不断地往左上角转移
状态转移概率矩阵 = numpy.zeros((最大生长高度 + 2, 最大生长高度 + 2))
状态转移概率矩阵[0, 0] = 状态转移概率矩阵[-1, -1] = 1
for i in range(最大生长高度):
    状态转移概率矩阵[1 + i, 0 + i] = p  # 每tick变化的概率
    状态转移概率矩阵[1 + i, 1 + i] = 1 - p  # 每tick不变的概率
    状态转移概率矩阵[1 + i, -1] = p  # 最后一列是产量
# print(状态转移概率矩阵)

总产量图 = [[]] * 26
总效率图 = [[]] * 26
总体积效率图 = [[]] * 26
计算刻数 = range(起始时间, 终止时间 + 1, 时间步长)
计算任务数 = 26
进度条 = tqdm(total=len(计算刻数) * 26)
最大线程数 = 26


def 计算(限高):  # 对于每一个限高
    # 生成该限高下的初始状态概率矩阵
    概率分布 = numpy.zeros((1, 27))  # 在最后有额外一个表示产量的数
    for 初始随机高度 in range(1, 26):  #
        概率分布[0, min(限高, 初始随机高度)] += 1 / len(range(1, 26))
    # print(概率分布)
    # 计算每个时间下的状态概率
    产量图 = []
    效率图 = []
    体积效率图 = []
    for tickedtime in 计算刻数:
        概率分布 = 概率分布 @ 状态转移概率矩阵
        # print(概率分布)
        产量 = 概率分布[0, -1]  # 第一行最后一列
        # 保存每一个时间的数据
        产量图.append(产量)
        效率图.append(产量 / (tickedtime + 收割时间))
        体积效率图.append(产量 / (tickedtime + 收割时间) / (限高 + 公摊高度))
        进度条.update()
    # 汇总每个限高的数据
    总产量图.append(产量图)
    总效率图.append(效率图)
    总体积效率图.append(体积效率图)
    # print(产量图)
    # print(效率图)
    # print(体积效率图)


with ThreadPoolExecutor(max_workers=min(计算任务数, 最大线程数)) as executor:
    计算任务 = [executor.submit(计算, 限高) for 限高 in range(1, 26)]
    wait(计算任务, return_when=ALL_COMPLETED)
    进度条.close()


#画图
pictureSize = 2160
plt.rcParams['font.size'] = pictureSize/100
print("产量图，效率图，体积效率图")
fig, axs = plt.subplots(3, 1, figsize=(pictureSize/100, 3*pictureSize/100))  # 2行2列的子图，调整 figsize 以适应你的需求
for 产量图 in 总产量图:
    axs[0].plot(range(len(产量图)), 产量图)
for 效率图 in 总效率图:
    axs[1].plot(range(len(效率图)), 效率图)
for 体积效率图 in 总体积效率图:
    axs[2].plot(range(len(体积效率图)), 体积效率图)

#坐标系网格线
for ax in axs:
    # 设置x轴的主刻度和次刻度
    ax.xaxis.set_major_locator(MultipleLocator(12000))
    ax.xaxis.set_minor_locator(MultipleLocator(2400))
    # 绘制x轴的网格线
    ax.grid(which='major', axis='x', linestyle='-', linewidth=1, color='#888888')
    ax.grid(which='minor', axis='x', linestyle=':', linewidth=0.5, color='#aaaaaa')

    # 绘制y轴的网格线（自动刻度）
    ax.grid(which='major', axis='y', linestyle='-', linewidth=1, color='#888888')
    ax.grid(which='minor', axis='y', linestyle=':', linewidth=0.5, color='#aaaaaa')


plt.tight_layout()
plt.savefig('my_figure.png', dpi=100, bbox_inches='tight')
plt.show()